<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Analytics/blob/main/Combine_Bath_Output_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from glob import glob
import re
import pandas as pd

file_names = [i for i in glob("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Deduplicated_Bath_Output/*_deduplicated.tbl")]

# Function to read a .tbl file into a DataFrame
def read_tbl_file_adjusted(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extracting the header
    header = lines[0].strip().split("\t")
    num_columns = len(header)

    # Processing the data rows
    data = []
    for line in lines[1:]:
        row = re.split(r'\s+', line.strip())
        if len(row) == num_columns:
            data.append(row)

    return pd.DataFrame(data, columns=header)

# Reading each file into a DataFrame and storing them in a list
dataframes = [read_tbl_file_adjusted(path) for path in file_names]

# Concatenating all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

print(combined_df.head())

# Convert e-value column to float for comparison
combined_df['E-value'] = combined_df['E-value'].astype(float)

# Group by 'contig name', 'ali from', and 'ali to', then keep the row with the minimum e-value
filtered_df = combined_df.loc[combined_df.groupby(['# target name', 'ali from', 'ali to'])['E-value'].idxmin()]

# Sort within each 'contig name' group by 'ali from'
sorted_df = filtered_df.groupby('# target name', group_keys=False).apply(lambda x: x.sort_values('ali from'))

# Save the sorted DataFrame to CSV and .tbl files
sorted_df.to_csv("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Final_Bath_Output/BATH_output_combined.csv", sep=",", index=False)
sorted_df.to_csv("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Final_Bath_Output/BATH_output_combined.tbl", sep="\t", index=False)

  # target name accession query name accession hmm len hmm from hmm to  \
0  k127_1013475         -     Q37556         -     334       12    325   
1  k127_1013475         -     Q4JQI2         -     227       61    223   
2  k127_1013475         -     Q35099         -     600        3    511   
3  k127_1013475         -     O47497         -     491       58    478   
4  k127_1013475         -     Q330A7         -     347        6    303   

  seq len ali from ali to env from env to  E-value  score  bias shifts stops  \
0  183529     1009   1989      991   1998  1.1e-57  205.2  22.6      0     0   
1  183529     4529   5035     4411   5049  5.2e-12   55.7   0.0      3     0   
2  183529    11371  12999    11365  13098  3.7e-78  272.2  35.3      0     0   
3  183529    13528  14802    13354  14826  1.5e-38  141.7  27.7      0     0   
4  183529    15321  16304    15302  16364  1.7e-15   69.1   0.0      0     1   

  pipe description of target  
0  std                     -  
1   fs      